In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"




['multiclass', 'key_a_d']

In [24]:
if not True:
    print('xxx')
# import os
# import glob


# class ImageProvider():
#     def __init__(self):
#         self.IMAGES_BASE_DIR = '/nfs/general/shared/images'
#         pass

#     def listImages(self, dataset):
#         if dataset not in os.listdir(self.IMAGES_BASE_DIR):
#             return {}
#         basedir=os.path.join(self.IMAGES_BASE_DIR, dataset)
#         classes = [(x, os.path.join(basedir, x)) for x in os.listdir(basedir)]
#         classes = filter(lambda x: os.path.isdir(x[1]), classes)
#         r = {}
#         for c in classes:
#             r[c[0]] = [x[len(self.IMAGES_BASE_DIR)+1:] for x in  glob.glob(c[1] + '/*.jpg')]
#         return r

#     def filePath(self, image_path):
#         image_path = os.path.normpath(os.path.join(self.IMAGES_BASE_DIR, image_path))
#         if os.path.commonprefix((image_path, self.IMAGES_BASE_DIR)) != self.IMAGES_BASE_DIR:
#             return None
#         return image_path


# ip=ImageProvider()

# ip.listImages('key_a_d')

# ip.filePath('key_a_d/KeyA/1650220626597_.jpg')

In [16]:
os.path.join('/nfs/general/shared/images', 'key_a_d/KeyA/1650220626597_.jpg')

'/nfs/general/shared/images/key_a_d/KeyA/1650220626597_.jpg'